In [1]:
pip install haversine

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [25]:
import pandas as pd
import requests
from haversine import haversine, Unit
from tqdm import tqdm

In [26]:
load_lst = pd.read_excel("주변식당목록.xlsx")
load_lst

,가맹점명,주소,분류,점수,건수
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27.0
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43.0
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2.0
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23.0
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16.0
...,...,...,...,...,...
385,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10.0
386,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3.0
387,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4.0
388,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4.0


In [27]:
# 내 위치 위,경도

st_x = 126.886336
st_y = 37.517872
print(st_y,st_x)
st = (float(st_y),float(st_x))

37.517872 126.886336


In [28]:
apiurl = "https://api.vworld.kr/req/address?"
params = {
	"service": "address",
	"request": "getcoord",
	"crs": "epsg:4326",
	"address": "서울 영등포구 선유로9길 30",
	"format": "json",
	"type": "road",
	"key": "79B583D6-112C-3703-8CB0-415188DD02E0"
}

response = requests.get(apiurl, params=params)
# if response.status_code == 200:
# 	print(response.json())

add_lst_b = []

for add in load_lst['주소']:
    
    # api에서 받아들일 수 있게 주소 재정의
    add_ = add.split()[0]+' '+add.split()[1]+' '+add.split()[2]+' '+add.split()[3]
    print(add_)
    params['address'] = add_
    response = requests.get(apiurl, params=params)

    try:
        en_x = response.json()['response']['result']['point']['x']
        en_y = response.json()['response']['result']['point']['y']
        print(f"x,y:{en_x,en_y}")
    except:
        # 위,경도 추출 실패한 주소 거리 비우고 추가
        add_lst_b.append(add)
        print(add)
        continue

    en = (float(en_y),float(en_x))
    dis = round(haversine(st,en,unit='m'),2) # 내 위치와 식당 거리 계산
    load_lst.loc[load_lst['주소']==add,'거리'] = dis

    print('='*50)

서울 영등포구 선유로9가길 16
x,y:('126.888335955', '37.518627664')
서울 영등포구 선유로17길 24
x,y:('126.887115333', '37.521002310')
서울 영등포구 선유로9길 10
x,y:('126.886983241', '37.517942848')
서울 영등포구 선유로17길 24
x,y:('126.887115333', '37.521002310')
서울 영등포구 선유로17길 24
x,y:('126.887115333', '37.521002310')
서울 영등포구 선유로13길 25
x,y:('126.886286204', '37.519045586')
서울 영등포구 문래북로 8
x,y:('126.885753503', '37.521342589')
서울 영등포구 선유로13길 25
x,y:('126.886286204', '37.519045586')
서울 영등포구 선유로17길 24
x,y:('126.887115333', '37.521002310')
서울 영등포구 선유서로 67
x,y:('126.884384857', '37.521729870')
서울 영등포구 선유로9길 10
x,y:('126.886983241', '37.517942848')
서울 영등포구 선유로13길 11
x,y:('126.887468675', '37.518988023')
서울 영등포구 선유로13길 25
x,y:('126.886286204', '37.519045586')
서울 영등포구 문래로 33
x,y:('126.885622558', '37.520633962')
서울 영등포구 문래북로 8
x,y:('126.885753503', '37.521342589')
서울 영등포구 선유로17길 24
x,y:('126.887115333', '37.521002310')
서울 영등포구 선유서로 35
x,y:('126.884419262', '37.518993074')
서울 영등포구 선유로9길 10
x,y:('126.886983241', '37.517942848')
서울 영등포구 

In [30]:
load_lst

,가맹점명,주소,분류,점수,건수,거리
0,화사랑화로구이 문래점,"서울 영등포구 선유로9가길 16 광양림빌딩 1,2층","육류,고기",3.5,27.0,195.38
1,라메흐꺄도,서울 영등포구 선유로17길 24 신일아르디세 1층 105~106호,"해물,생선",4.1,43.0,354.80
2,화덕으로간고등어반상,서울 영등포구 선유로9길 10 문래SKV1센터 1층 119호,"해물,생선",4.5,2.0,57.62
3,쁘띠문래,서울 영등포구 선유로17길 24 신일아르디세 1층 109호,양식,4.1,23.0,354.80
4,문래옥,서울 영등포구 선유로17길 24 1층,"육류,고기",4.1,16.0,354.80
...,...,...,...,...,...,...
385,설봄,서울 영등포구 선유서로 17 1층 101호,"떡,한과",4.3,10.0,224.20
386,김가네 문래SKV1점,서울 영등포구 선유로9길 10 1층 108호,분식,2.3,3.0,57.62
387,라라커피,서울 영등포구 선유로9나길 12 문래 두산위브 상가동 1층 105호,카페,4.8,4.0,141.83
388,피자알볼로 문래점,서울 영등포구 선유로3길 10 하우스디비즈 1층 101호,피자,3.8,4.0,166.52


In [31]:
load_lst.to_excel("거리추가.xlsx", index=False)